In [56]:
from datetime import datetime
import pandas as pd
from openbb import obb
import os

obb.user.credentials.fmp_api_key="edc2977d918bfc0e71920ca88e986b39"
data_folder = 'data'
company = 'SPY'
out_path = os.path.join(data_folder, company)
interval = '1d' # 1m, 1h, 1d options
start_date = '2014-01-01'
end_date = '2024-01-01'
df = obb.equity.price.historical(company, interval, start_date, end_date, provider="fmp").to_df()
df

,open,high,low,close,volume,vwap,label,adj_close,unadjusted_volume,change,change_percent,change_over_time
date,,,,,,,,,,,,
2014-01-02,183.98,184.07,182.48,182.92,119636900,183.16,"January 02, 14",152.67,119636900.0,-1.06,-0.576150,-0.005762
2014-01-03,183.23,183.60,182.63,182.89,81390600,183.04,"January 03, 14",152.64,81390600.0,-0.34,-0.185560,-0.001856
2014-01-06,183.49,183.56,182.08,182.36,108028200,182.67,"January 06, 14",152.20,108028200.0,-1.13,-0.615840,-0.006158
2014-01-07,183.09,183.79,182.95,183.48,86144200,183.41,"January 07, 14",153.14,86144200.0,0.39,0.213010,0.002130
2014-01-08,183.45,183.83,182.89,183.52,96582300,183.41,"January 08, 14",153.17,96582300.0,0.07,0.038158,0.000382
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,473.86,475.38,471.70,473.65,67160419,473.80,"December 22, 23",473.65,67126600.0,-0.21,-0.044317,-0.000443
2023-12-26,474.07,476.58,473.99,475.65,55386952,475.11,"December 26, 23",475.65,55387000.0,1.58,0.333280,0.003333
2023-12-27,475.44,476.66,474.89,476.51,67999311,475.77,"December 27, 23",476.51,68000300.0,1.07,0.225050,0.002250


In [57]:
df2 = obb.equity.price.historical(company, interval, start_date, end_date, provider="yfinance").to_df()
df2

,open,high,low,close,volume,dividends,stock_splits,capital_gains
date,,,,,,,,
2014-01-02,183.98,184.07,182.48,182.92,119636900,0.0,0.0,0.0
2014-01-03,183.23,183.60,182.63,182.89,81390600,0.0,0.0,0.0
2014-01-06,183.49,183.56,182.08,182.36,108028200,0.0,0.0,0.0
2014-01-07,183.09,183.79,182.95,183.48,86144200,0.0,0.0,0.0
2014-01-08,183.45,183.83,182.89,183.52,96582300,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2023-12-22,473.86,475.38,471.70,473.65,67126600,0.0,0.0,0.0
2023-12-26,474.07,476.58,473.99,475.65,55387000,0.0,0.0,0.0
2023-12-27,475.44,476.66,474.89,476.51,68000300,0.0,0.0,0.0


In [59]:
df2["pct_change"] = df2["close"].pct_change() * 100
df2

,open,high,low,close,volume,dividends,stock_splits,capital_gains,pct_change
date,,,,,,,,,
2014-01-02,183.98,184.07,182.48,182.92,119636900,0.0,0.0,0.0,NaN
2014-01-03,183.23,183.60,182.63,182.89,81390600,0.0,0.0,0.0,-0.016401
2014-01-06,183.49,183.56,182.08,182.36,108028200,0.0,0.0,0.0,-0.289792
2014-01-07,183.09,183.79,182.95,183.48,86144200,0.0,0.0,0.0,0.614170
2014-01-08,183.45,183.83,182.89,183.52,96582300,0.0,0.0,0.0,0.021801
...,...,...,...,...,...,...,...,...,...
2023-12-22,473.86,475.38,471.70,473.65,67126600,0.0,0.0,0.0,0.200973
2023-12-26,474.07,476.58,473.99,475.65,55387000,0.0,0.0,0.0,0.422253
2023-12-27,475.44,476.66,474.89,476.51,68000300,0.0,0.0,0.0,0.180805
